<h1><center> <span style="color:blue">Segmenting and Clustering Neighborhoods in Toronto<br> Part II </span><h1>
    

<h3><center>The purpose of this notebook is to scrape code from a Wikipedia page, about Toronto Neighbourhoods. Then put it into a Dataframe. <br> After this the Longtitude and Latitude using the Geocoder Python package<center><h3>

In [15]:
#imports
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from requests import get 
import lxml.html as lh
import geocoder

In [16]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'           #the website which we are using for our data 
response=get(url)                                                               #request the content of the webpage from the server and store it in the variable 'respose'

In [17]:
doc = lh.fromstring(response.content)                                           #storing the contents of website in a document
tr_elements = doc.xpath('//tr')                                                 #extracting the data stored between <tr></tr> tags (which define a table)
#create an empty list to store the table data in 
col=[]
i=0
#stroing the headers
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))
    
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    #this is to ensure that each row has the same length
    if len(T)!=3:
        break
    i=0
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #if there is any numerical data convert it into integers 
            try:
                data=int(data)
            except:
                pass
        #Append the data
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [18]:
#create the dataframe 
Dict={title:column for (title,column) in col}
df_neighbourhood=pd.DataFrame(Dict)
print(df_neighbourhood.shape)
df_neighbourhood.head()

(287, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


 #### DATA CLEANING

In [19]:
#removing all the columns which do not have an assigned Borough
df_neighbourhood=df_neighbourhood[df_neighbourhood.Borough!="Not assigned"]
print(df_neighbourhood.shape)
df_neighbourhood=df_neighbourhood.reset_index(drop=True)       #reset the index 
df_neighbourhood.rename(columns={'Neighbourhood\n':'Neighbourhood'},inplace=True)    #renaming column to replace '\n' character
df_neighbourhood['Neighbourhood'] = df_neighbourhood['Neighbourhood'].map(lambda x: x.rstrip('\n'))   #removing the '\n' that appears after the neighbourhoods
df_neighbourhood.head()

(210, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [20]:
df_neighbourhood=df_neighbourhood.groupby('Postcode').agg(lambda x: ','.join(set(x)))   #groupby postcode and combine rows with same postcode 
df_neighbourhood=df_neighbourhood.reset_index(drop=False)
df_neighbourhood.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"West Hill,Guildwood,Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
#if a cell as an assigned Borough but not neighbourhood then the burough becomes thw neighbourhood 
df_neighbourhood['Neighbourhood']=np.where(df_neighbourhood['Neighbourhood']=='Not assigned',df_neighbourhood['Borough'],df_neighbourhood['Neighbourhood'])

In [22]:
df_neighbourhood.shape

(103, 3)

In [26]:
#downlaod longtitude and latitude data 
!wget -q -O 'longandlat_data.csv' https://cocl.us/Geospatial_data
long_lat_df=pd.read_csv('longandlat_data.csv')
long_lat_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [32]:
#append the logtitude and latitude columns to df_neighbourhood 
df_neighbourhood['Latitude']=long_lat_df['Latitude']
df_neighbourhood['Longitude']=long_lat_df['Longitude']
df_neighbourhood.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill,Guildwood,Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
